In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html
#!pip install wandb
#!wandb login

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 220.5 MB 42 kB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 281 kB 74.9 MB/s 
     |████████████████████████████████| 103 kB 78.1 MB/s 
     |████████████████████████████████| 596 kB 73.6 MB/s 
     |████████████████████████████████| 109 kB 93.5 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
     |████████████████████████████████| 51 kB 522 kB/s 
     |████████████████████████████████| 11.1 MB 52.6 MB/s 
     |████████████████████████████████| 42 kB 926 kB/s 
     |████████████████████████████████| 3.1 MB 74.8 MB/s 
     |████████████████████████████████| 546 kB 10.4 MB/s 
     |████████████████████████████████| 121 kB 81.5 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |███████████

In [1]:
%ls
%load_ext autoreload
%autoreload 2

drive/  sample_data/


In [2]:
%cd drive
%cd MyDrive
%cd SIG-VAE-GIN-and-fMRI
%cd SIG-VAE-GIN-and-fMRI
from sigvaemodel import *
from loss import *

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI


In [3]:
%reload_ext autoreload
from sigvaemodel import *
from loss import loss as loss_function
#from loss import get_rec

In [4]:
import numpy as np
import torch
import torch.nn as nn
import dgl.data
import networkx as nx
import scipy.sparse as sp
from sklearn.metrics import average_precision_score, roc_curve, precision_recall_curve, roc_auc_score
import sigvaemodel
import loss
import time
import wandb
import torch.optim as optim
import gc
from matplotlib import pyplot


CoraDataset=dgl.data.CoraGraphDataset(reverse_edge=False)
graph=CoraDataset[0]
u, v=graph.edges()
print(len(u))
featureMatrix=graph.ndata['feat']
featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1])
print(featureMatrix.size())
print(featureMatrix)
graph.to('cuda')
config=dict(
    learning_rate=0.0001,
    architecture="GIN",
    dataset="Cora",
    noise_dim=32,
    warming_up=100,
)


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
10556
torch.Size([1, 2708, 1433])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]])


In [5]:
def edges_to_lists(headMatrix, tailMatrix):
    results=[]
    assert len(headMatrix)==len(tailMatrix)
    hlength=len(headMatrix)
    for i in range(hlength):
        results.append([headMatrix[i], tailMatrix[i]])
    return results

In [6]:
def preprocess(graph):
    N=graph.num_nodes()
    numE=graph.number_of_edges()
    halfnumE=int(numE/2)
    
    u, v=graph.edges()
    #graph.add_edges(v, u, graph.edata)
    #u, v=graph.edges()
    #numEtimes2=graph.number_of_edges()
    #print(numEtimes2)
    idx=np.arange(halfnumE)
    testidx=np.random.permutation(idx)
    testSize=int(len(idx)/10)
    validation_size=int(len(idx)/20)
    train_size=numE-testSize-validation_size
    '''
    adj1=sp.coo_matrix((np.ones(len(u)), (u.detach().cpu().numpy(), v.detach().cpu().numpy())))
    adj2=sp.coo_matrix((np.ones(len(u)), (v.detach().cpu().numpy(), u.detach().cpu().numpy())))
    adj=adj1+adj2
    adj=torch.Tensor(adj.todense())
    print(adj)
    adj[adj==2]=1
    '''
    adj=sp.coo_matrix((np.ones(len(u)), (u.detach().cpu().numpy(), v.detach().cpu().numpy())))
    adj=torch.Tensor(adj.todense())
    adj_upper=torch.triu(adj, diagonal=1)
    adj_lower=torch.tril(adj, diagonal=-1)

    uUpper, vUpper=torch.where(adj_upper!=0)
    print("^^^^^^^^^^^^^^^^6666")
    print(uUpper.shape)
    print(vUpper.shape)
    test_pos_u, test_pos_v=uUpper[testidx[:testSize]], vUpper[testidx[:testSize]]
    train_pos_u, train_pos_v = uUpper[testidx[testSize:halfnumE-validation_size]], vUpper[testidx[testSize:halfnumE-validation_size]]
    validation_pos_u, validation_pos_v=uUpper[testidx[halfnumE-validation_size:]], vUpper[testidx[halfnumE-validation_size:]]
    
    #print("&&&&&&&&&&&&&&&&&&&&&")
    #print(adj.shape)
    #print(torch.max(adj))
    adj_neg_upper=torch.triu(1-adj_upper, diagonal=1)
    neg_u, neg_v=torch.where(adj_neg_upper!=0)
    #print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    #print(len(neg_u))
    #print(len(neg_v))
    neg_testidx=np.random.choice(len(neg_u), halfnumE)
    test_neg_u, test_neg_v=neg_u[neg_testidx[:testSize]], neg_v[neg_testidx[:testSize]]
    train_neg_u, train_neg_v=neg_u[neg_testidx[testSize:halfnumE-validation_size]], neg_v[neg_testidx[testSize:halfnumE-validation_size]]
    validation_neg_u, validation_neg_v=neg_u[neg_testidx[halfnumE-validation_size:]], neg_v[neg_testidx[halfnumE-validation_size:]]
    #print(test_neg_u)
    #print(test_neg_v)
    
    adj_train1=sp.coo_matrix((np.ones(len(train_pos_u)), (train_pos_u.detach().cpu().numpy(), train_pos_v.detach().cpu().numpy())),shape=(N, N))
    adj_train2=sp.coo_matrix((np.ones(len(train_pos_u)), (train_pos_v.detach().cpu().numpy(), train_pos_u.detach().cpu().numpy())),shape=(N, N))
    adj_train=adj_train1+adj_train2
    #adj_train[adj_train==2]=1
    #adj_train=adj_train-sp.dia_matrix((adj_train.diagonal()[np.newaxis, :], [0]), shape=adj_train.shape)
    adj_train=torch.Tensor(adj_train.todense())
    print(torch.max(adj_train))
    
    adj_test1=sp.coo_matrix((np.ones(len(test_pos_u)), (test_pos_u.detach().cpu().numpy(), test_pos_v.detach().cpu().numpy())),shape=(N,N))
    adj_test2=sp.coo_matrix((np.ones(len(test_pos_u)), (test_pos_v.detach().cpu().numpy(), test_pos_u.detach().cpu().numpy())),shape=(N,N))
    adj_test=adj_test1+adj_test2
    #adj_test[adj_test==2]=1
    #adj_test=adj_test-sp.dia_matrix((adj_test.diagonal()[np.newaxis, :], [0]), shape=adj_test.shape)
    adj_test=torch.Tensor(adj_test.todense())
    print(torch.max(adj_test))

    adj_validation1=sp.coo_matrix((np.ones(len(validation_pos_u)), (validation_pos_u.detach().cpu().numpy(), validation_pos_v.detach().cpu().numpy())),shape=(N,N))
    adj_validation2=sp.coo_matrix((np.ones(len(validation_pos_u)), (validation_pos_v.detach().cpu().numpy(), validation_pos_u.detach().cpu().numpy())),shape=(N,N))
    adj_validation=adj_validation1+adj_validation2
    #adj_validation[adj_validation==2]=1
    #adj_validation=adj_validation-sp.dia_matrix((adj_validation.diagonal()[np.newaxis, :], [0]), shape=adj_validation.shape)
    adj_validation=torch.Tensor(adj_validation.todense())
    print(torch.max(adj_validation))


    adj_train_neg1=sp.coo_matrix((np.ones(len(train_neg_u)), (train_neg_u, train_neg_v)),shape=(N, N))
    adj_train_neg2=sp.coo_matrix((np.ones(len(train_neg_u)), (train_neg_v, train_neg_u)),shape=(N, N))
    adj_train_neg=adj_train_neg1+adj_train_neg2
    
    #adj_train=adj_train-sp.dia_matrix((adj_train.diagonal()[np.newaxis, :], [0]), shape=adj_train.shape)
    adj_train_neg=torch.Tensor(adj_train_neg.todense())
    adj_train_neg[adj_train_neg==2]=1
    print(torch.max(adj_train_neg))
    
    adj_test_neg1=sp.coo_matrix((np.ones(len(test_neg_u)), (test_neg_u, test_neg_v)),shape=(N,N))
    adj_test_neg2=sp.coo_matrix((np.ones(len(test_neg_u)), (test_neg_v, test_neg_u)),shape=(N,N))
    adj_test_neg=adj_test_neg1+adj_test_neg2
    #adj_test[adj_test==2]=1
    #adj_test=adj_test-sp.dia_matrix((adj_test.diagonal()[np.newaxis, :], [0]), shape=adj_test.shape)
    adj_test_neg=torch.Tensor(adj_test_neg.todense())
    print(torch.max(adj_test_neg))

    adj_validation_neg1=sp.coo_matrix((np.ones(len(validation_neg_u)), (validation_neg_u, validation_pos_v)),shape=(N,N))
    adj_validation_neg2=sp.coo_matrix((np.ones(len(validation_neg_u)), (validation_neg_v, validation_pos_u)),shape=(N,N))
    adj_validation_neg=adj_validation_neg1+adj_validation_neg2
    #adj_validation[adj_validation==2]=1
    #adj_validation=adj_validation-sp.dia_matrix((adj_validation.diagonal()[np.newaxis, :], [0]), shape=adj_validation.shape)
    adj_validation_neg=torch.Tensor(adj_validation_neg.todense())
    print(torch.max(adj_validation_neg))


    train_u, train_v=torch.where(adj_train!=0)
    train_graph=dgl.graph((torch.hstack((train_u, train_v)), torch.hstack((train_v, train_u))))
    #train_graph=dgl.remove_edges(graph, np.hstack((testidx[:testSize],testidx[numE-validation_size:], (testidx[:testSize]+numE), (testidx[numE-validation_size:]+numE))))
    #train_graph=dgl.remove_edges(train_graph_temp, np.hstack((testidx[:testSize], testidx[numE-validation_size:]+numE)))
    test_u, test_v=torch.where(adj_test!=0)
    test_graph=dgl.graph((torch.hstack((test_u, test_v)), torch.hstack((test_v, test_u))))
    #test_graph=dgl.remove_edges(graph, np.hstack((testidx[testSize:], (testidx[testSize:]+numE))))
    #test_graph=dgl.remove_edges(test_graph_temp, testidx[testSize:]+numE)
    validation_u, validation_v=torch.where(adj_validation!=0)
    validation_graph=dgl.graph((torch.hstack((validation_u, validation_v)), torch.hstack((validation_v, validation_u))))
    #validation_graph=dgl.remove_edges(graph, np.hstack((testidx[:numE-validation_size], (testidx[:numE-validation_size]+numE))))
    #validation_graph=dgl.remove_edges(validation_graph_temp ,testidx[:numE-validation_size]+numE)
    
    train_neg_u, train_neg_v=torch.where(adj_train_neg!=0)
    test_neg_u, test_neg_v=torch.where(adj_test_neg!=0)
    validation_neg_u, validation_neg_v=torch.where(adj_validation_neg!=0)
    '''
    train_pos=edges_to_lists(train_pos_u, train_pos_v)
    test_pos=edges_to_lists(test_pos_u, test_pos_v)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)
    validation_pos=edges_to_lists(validation_pos_u, validation_pos_v)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)
    '''
    '''
    train_pos=edges_to_lists(train_pos_u, train_pos_v)+edges_to_lists(train_pos_v, train_pos_u)
    test_pos=edges_to_lists(test_pos_u, test_pos_v)+edges_to_lists(test_pos_v, test_pos_u)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)+edges_to_lists(test_neg_v, test_neg_u)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)+edges_to_lists(train_neg_v, train_neg_u)
    validation_pos=edges_to_lists(validation_pos_u, validation_pos_v)+edges_to_lists(validation_pos_v, validation_pos_u)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)+edges_to_lists(validation_neg_v, validation_neg_u)
    '''
    train_pos=edges_to_lists(train_u, train_v)
    test_pos=edges_to_lists(test_u, test_v)
    validation_pos=edges_to_lists(validation_u, validation_v)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)

    print(len(validation_pos))
    print(len(validation_neg))
    featureMatrix=graph.ndata['feat']
    featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1]).to('cuda')


    return graph, adj, adj_train, adj_validation, adj_test, featureMatrix, train_graph, validation_graph, test_graph, train_pos, validation_pos, test_pos, train_neg, validation_neg, test_neg


In [7]:
def step(model, adj, adj_label, feat, input_graph, epoch, K, J, weight, norm, dropout, optimizer, scheduler, device):
  #global loss
    if optimizer is None:
        with torch.no_grad():
          model.eval()
    else:
        model.train()
    #WU=torch.min(torch.Tensor([epoch/500., 1.]))
    WU=1
    
    #generated_prob, mu, sigma, latent_representation, Z, epsilon=model.forward(adj.to(device), feat.to(device), input_graph)
    generated_prob, mu, sigma, latent_representation, Z, epsilon, rk=model.forward(adj.to(device), feat.to(device), input_graph)
    loss=loss_function(generated_prob, adj_label, mu, sigma, Z, epsilon, latent_representation, K, J, WU, norm, weight, dropout, device=device)
    
    #loss_rec, loss_prior, loss_post=loss_function(generated_prob, adj_label, mu, sigma, Z, epsilon, latent_representation, K, J, norm, weight, dropout, device=device)
    #reg=(loss_post-loss_prior) / (adj.size(0)**2)
    #loss=loss_rec+reg
    
    #print(generated_prob.shape)
    #print(adj.shape)
    #loss=loss_rec+loss_prior+loss_post
    
    #WarmingUP=np.min([epoch/300,1])
    #reg=(loss_post-loss_prior) * WarmingUP / (adj.size(0)**2)
    #loss=loss_rec+WarmingUP*reg
    
    
    if optimizer is not None:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
          scheduler.step()
    del mu
    del sigma
    del epsilon
    return latent_representation, generated_prob, loss, Z, rk

In [8]:
def get_auc(embedding, adj_orig, rk, pos_array, neg_array, decoder_type):
    
    def dc(x):
      if decoder_type=="ip":
        return 1/(1+torch.exp(-x))
      elif decoder_type=="bp":
        return 1-torch.exp(-torch.exp(x))
    
    J=embedding.shape[0]
    rk_expand=torch.tile(torch.diag(rk), (J, 1, 1))
    X=torch.transpose(embedding, 1, 2)
    X=torch.bmm(rk_expand, X)
    X=torch.bmm(embedding, X)
    X=torch.clamp(X, min=float('-inf'), max=10)
    adj_rec=1-torch.exp(-torch.exp(X))
    #print("@@@@@@@@@@@@@@@@@@@@")
    #print(adj_rec)
    '''
    adj_rec=torch.zeros(J, embedding.shape[1], embedding.shape[1])
    for i, emb in enumerate(embedding):
      tmp = torch.matmul(torch.diag(rk), emb.t())
      adj_rec_sub = 1 - torch.exp(- torch.exp(torch.matmul(emb, tmp)))
      adj_rec[i]=adj_rec_sub
    '''
    preds_pos=torch.zeros(J, pos_array.shape[1])
    for i, e in enumerate(pos_array.t()):
      preds_pos[:,i]=adj_rec[:,e[0], e[1]]
    
    preds_neg=torch.zeros(J, neg_array.shape[1])
    for i, e in enumerate(neg_array.t()):
      preds_neg[:,i]=adj_rec[:, e[0], e[1]]

    #preds_pos=adj_rec[:, pos_array[0], pos_array[1]]
    #preds_neg=adj_rec[:, neg_array[0], neg_array[1]]
    '''
    emb_pos_head=embedding[:, pos_array[0], :]
    emb_pos_tail=embedding[:, pos_array[1], :]
    preds_pos=dc(torch.einsum('abc, abc->ab', emb_pos_head, emb_pos_tail))
    emb_neg_head=embedding[:, neg_array[0], :]
    emb_neg_tail=embedding[:, neg_array[1], :]
    preds_neg=dc(torch.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail))
    #preds_neg=torch.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail)
    '''

    preds=torch.hstack([preds_pos, preds_neg]).detach().cpu().numpy()
    print(preds)
    labels=torch.hstack([torch.ones(preds_pos.shape[-1]), torch.zeros(preds_neg.shape[-1])]).detach().cpu().numpy()
    #print(preds[0])
    #print(preds[1])
    roc=torch.Tensor([roc_auc_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    avg_precision=torch.Tensor([average_precision_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    '''
    fpr_accumulate=0
    tpr_accumulate=0
    recall_accumulate=0
    precision_accumulate=0
    for pred in np.vsplit(preds,J):
      fpr, tpr, _=roc_curve(labels, pred.flatten())
      precision, recall, _=precision_recall_curve(labels, pred.flatten())
      fpr_accumulate+=fpr.mean()
      tpr_accumulate+=tpr.mean()
      recall_accumulate+=recall.mean()
      precision_accumulate+=precision.mean()
    fpr_mean=fpr_accumulate/J
    tpr_mean=tpr_accumulate/J
    recall_mean=recall_accumulate/J
    precision_mean=precision_accumulate/J
    '''
    #precision, recall=torch.Tensor([precision_recall_curve(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])])
    #accuracy=torch.Tensor([accuracy_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #precision=torch.Tensor([precision_score(labels, pred) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #recall=torch.Tensor([recall_score(labels, pred) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #precision=precision.mean()
    #recall=recall.mean()
    #fpr=fpr.mean()
    del adj_rec
    del preds_pos
    del preds_neg
    del preds
    del labels

    return roc, avg_precision #, precision_mean, recall_mean, fpr_mean, tpr_mean

In [9]:
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

import wandb

wandb.config = {
  "learning_rate": 0.001,
  "epochs": 5000,
  "batch_size": 35,
  "K" : 15,
  "J" : 20, 
  #"WU" : 500,
  "noise_dim" : 64
}
#wandb.init(project="SIG-VAE-GIN-and-fMRI", entity="ytrewq271828", config=config)

graph_processed, adj, adj_train, adj_validation, adj_test, featureMatrix, train_graph,validation_graph, test_graph, train_pos, validation_pos, test_pos, train_neg, validation_neg, test_neg=preprocess(graph)
adj_label=adj_train+torch.eye(adj_train.shape[0])
adj_label=torch.FloatTensor(adj_label)

#print((adj.transpose(0,1)==adj).all())
##print((adj_train.transpose(0,1)==adj_train).all())
#print((adj_test.transpose(0,1)==adj_test).all())
#print((adj_validation.transpose(0,1)==adj_validation).all())
adj=torch.Tensor(adj).to(device)
adj_test=torch.Tensor(adj_test).to(device)
featureMatrix=torch.cuda.FloatTensor(featureMatrix)
adj_train=torch.Tensor(adj_train).to(device)
train_graph=train_graph.to(device)
test_graph=test_graph.to(device)
validation_graph=validation_graph.to(device)
graph_processed=graph_processed.to(device)
N=adj.shape[0]
weight=torch.tensor([float(N * N - adj.sum()) / adj.sum()]).to(device)
norm=N*N/float((N*N-adj.sum())*2)

pos_array_train=torch.cuda.LongTensor(train_pos).t() #(2, #positive edges)
neg_array_train=torch.cuda.LongTensor(train_neg).t() #(2, #negative edges)
pos_array_test=torch.cuda.LongTensor(test_pos).t()
neg_array_test=torch.cuda.LongTensor(test_neg).t()
pos_array_validation=torch.cuda.LongTensor(validation_pos).t()
neg_array_validation=torch.cuda.LongTensor(validation_neg).t()
#print(torch.count_nonzero(adj_train))
#print(torch.count_nonzero(adj_test))
#print(adj_test.shape)
#print(pos_array_train)
print(pos_array_validation.shape)
print(neg_array_validation.shape)

torch.autograd.set_detect_anomaly(False)

model=SIGVAE_GIN(Lu=1, 
                Lmu=1, 
                Lsigma=1, 
                input_dim=featureMatrix.shape[2], 
                output_dim_u=[32], 
                output_dim_mu=[16], 
                output_dim_sigma=[16],
                K=15,
                J=20,
                device=device,
                noise_dim=64,
                decoder_type="bp",
                activation=torch.nn.functional.relu,
                dropout=0)
K=15
J=20
dropout=0
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0, verbose=True)
epoch=5000
loss_accumulate=0
for epc in range(epoch):
    t=time.time()
    model.train()
    latent_representation, generated_prob, loss_train, z_proc, rk=step(model, adj_train, adj_label, featureMatrix, train_graph, epc, K, J, weight, norm, dropout, optimizer=optimizer, scheduler=scheduler, device=device)
    loss_accumulate+=loss_train
    #print(z_proc)    
    #print(z_proc.shape)
    #z_proc=z_proc.detach().cpu().numpy()
    
    #roc, avg_precision, precision, recall, fpr, tpr=get_auc(z_proc, pos_array_validation, neg_array_validation, decoder_type="bp")
    roc, avg_precision=get_auc(z_proc, adj, rk,  pos_array_validation, neg_array_validation, decoder_type="bp")
    
    #roc, acc, prc, rcl=get_auc(z_proc, pos_array_train, neg_array_train, decoder_type="bp")
    #print(loss_train.item())
    #print(roc)
    print("Epoch : ", '%d' %(epc+1), "loss=", "%f" %(loss_train.item()),
          "time=", "%f" %(time.time()-t), "roc=", "%f" %(roc), "average_precision", "%f" %(avg_precision),
          #"precision", "%f" %(precision), "recall", "%f" %(recall),
          #"false positive rate", "%f" %(fpr), "true positive rate", "%f" %(tpr))
    )
    #pyplot.plot(fpr, recall, marker='.', label='roc-auc')
    
    '''
    wandb.log({"loss_train": loss_train})
    wandb.log({"roc-auc": roc})
    wandb.log({"average_precision": avg_precision})
    wandb.watch(model)
    '''
    '''
    wandb.log({"recall_train" : recall})
    wandb.log({"fpr_train" : fpr})
    wandb.watch(model)
    '''
    del latent_representation
    del generated_prob
    del z_proc
    del roc
    del rk
    del avg_precision
    gc.collect()
    torch.cuda.empty_cache()
    if (epc+1) % 20 == 0:
          with torch.no_grad():
            model.eval()
            #generated_prob, mu, sigma, latent_representation, z_proc, epsilon=model(adj_test, featureMatrix, test_graph)
            latent_representation, generated_prob, loss_test, z_proc, rk=step(model, adj_train, adj_label, featureMatrix, train_graph,  epc, K, J, weight, norm, dropout, optimizer=None, scheduler=None, device=device)

            #z_proc = z_proc.detach().cpu().numpy()
            roc, avg_pre= get_auc(z_proc, adj, rk, pos_array_test, neg_array_test, decoder_type="bp")
            rslt = "Test ROC score: %f" %(roc)
            rslt2="Test Precision score : %f" %(avg_pre)
            print("\n", rslt, "  ", rslt2, "\n")
            '''
            wandb.log({"loss_test": loss_test})
            wandb.log({"roc-auc_test" : roc})
            wandb.watch(model)
            '''
            del latent_representation
            del generated_prob
            del z_proc
            del roc
            del rk
            #del avg_precision
            gc.collect()
            torch.cuda.empty_cache()
#pyplot.show()
    #latent_representation, generated_prob, loss_train=step(model, adj_train, epoch, optimizer )

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 [0.6626122  0.6718938  0.6626122  ... 0.67231655 0.6807251  0.84159243]
 ...
 [0.58559453 0.633976   0.58559453 ... 0.81350493 0.5797703  0.65106577]
 [0.5864724  0.57295716 0.5864724  ... 0.6518227  0.6693058  0.66958886]
 [0.59364843 0.5972827  0.59364843 ... 0.28206527 0.7064314  0.74406826]]
Epoch :  224 loss= 0.757087 time= 1.006857 roc= 0.539885 average_precision 0.541650
Adjusting learning rate of group 0 to 5.0000e-04.
[[0.6842732  0.66490114 0.6842732  ... 0.6651579  0.4509235  0.7467973 ]
 [0.5727029  0.6909356  0.5727029  ... 0.7658833  0.45872605 0.76840335]
 [0.7336098  0.6010189  0.7336098  ... 0.5909201  0.52312076 0.6143621 ]
 ...
 [0.64272827 0.62032354 0.64272827 ... 0.6143278  0.5760101  0.8658499 ]
 [0.58001065 0.5640532  0.58001065 ... 0.6211989  0.6457106  0.5415344 ]
 [0.5883035  0.57630086 0.5883035  ... 0.6492869  0.58802736 0.57055366]]
Epoch :  225 loss= 0.757737 time= 0.997612 roc= 0.548590 average_precision 0.557729
Adju

KeyboardInterrupt: ignored

In [ ]:
  !nvidia-smi

Mon May 23 04:04:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |  11124MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------